In [ ]:
# Logic programming

In [46]:
# Matching mathematical expressions
from kanren import run, var, fact
import kanren.assoccomm as la
from kanren.assoccomm import eq_assoccomm as eq

# Define mathematical operations
add = 'addition'
mul = 'multiplication'

# Declare that these operations are commutative
# using facts system
fact(la.commutative, mul)
fact(la.commutative, add)
fact(la.associative, mul)
fact(la.associative, add)

# Define some variables
a, b, c = var('a'), var('b'), var('c')

expression_orig = 3 * (-2) + (1 + 2 * 3) * (-1)

# Generate expressions
expression_orig = (add, (mul,3,-2), (mul, (add, 1, (mul,2,3)), -1))
expression1 = (add, (mul,(add,1,(mul,2,a)),b), (mul,3,c))
expression2 = (add, (mul,c,3), (mul,b, (add, (mul,2,a),1)))
expression3 = (add, (add, (mul, (mul,2,a), b), b),(mul,3,c))

print(run(0, (a,b,c), eq(expression1, expression_orig)))
print(run(0, (a,b,c), eq(expression2, expression_orig)))
print(run(0, (a,b,c), eq(expression3, expression_orig)))

((3, -1, -2),)
((3, -1, -2),)
()


In [ ]:
# Three values in the first two above lines represents the values for a,b,c 
# The first two expessions matches the original expression (expression_orig)
# The third one returns nothing because it is structurally different.

In [47]:
from kanren import isvar,run,membero
from kanren.core import success,fail,goaleval,condeseq,eq,var
from sympy.ntheory.generate import prime,isprime
import itertools as it
def prime_test(n): #Function to test for prime
    if isvar(n):
        return condeseq([(eq,n,p)] for p in map(prime,it.count(1)))
    else:
        return success if isprime(n) else fail
n=var() #Variable to use
set(run(0,n,(membero,n,(12,14,15,19,21,20,22,29,23,30,41,44,62,52,65,85)),(prime_test,n)))

{19, 23, 29, 41}

In [51]:
# validating primes
from kanren import isvar,run,membero
from kanren.core import success,fail,goaleval,condeseq,eq,var

import itertools as it
import kanren.core as lc
from sympy.ntheory.generate import prime, isprime
from kanren import eq, membero, var, conde, isvar 
from kanren.core import condeseq, fail, run

# Check if the elements of x are prime
def check_prime(x):
    if isvar(x):
        return condeseq([(eq, x, p)] for p in map(prime, it.count(1)))
    else: 
        return success if isprime(x) else fail
    
# declare the variable
x = var()

# Check if a number in a list is a prime number
# membero checks if a given number is a member of the list of numbers specified in the input argument
list_nums = (23,4, 27, 17, 13, 10, 21, 29, 3, 32, 11, 19)
print('\n List of prime numbers:')
print(set(run(0, x, (membero, x, list_nums), (check_prime,x))))

# Print first 7 prime numbers
print('\n List of first 7 prime numbers:')
print(run(7, x, check_prime(x)))








 List of prime numbers:
{3, 11, 13, 17, 19, 23, 29}

 List of first 7 prime numbers:
(2, 3, 5, 7, 11, 13, 17)


In [46]:
import json
from kanren import Relation, facts, run, conde, var, eq

# check if 'x' is the parent of 'y'
def parent(x,y):
    return conde([father(x,y)], [mother(x,y)])

# Check if 'x' is the grandparents of 'y'
def grandparent(x,y):
    temp = var()
    return conde((parent(x, temp), parent(temp,y)))

# Check for sibling relationship bw. 'x' and 'y'
def sibling(x,y):
    temp = var()
    return conde((parent(temp,x), parent(temp,y)))
    
# check if x is y's uncle
def uncle(x,y):
    temp = var()
    return conde((father(temp,x), grandparent(temp,y)))

if __name__ == '__main__':
    father = Relation()
    mother = Relation()
    
    with open('C:/Users/Peyman/Desktop/Python - Practices/relationships.json') as f:
        d = json.loads(f.read())
        
    for item in d['father']:
        facts(father, (list(item.keys())[0], list(item.values())[0]))
    for item in d['mother']:
        facts(mother, (list(item.keys())[0], list(item.values())[0]))
        
    x = var()
    
    # John's children
    name = 'John'
    output = run(0,x,father(name,x))
    print("\n List of "+name+ "'s children:'")
    for item in output:
        print(item)
        
    # William's mother
    name = 'William'
    output = run(0, x, mother(x,name))[0]
    print("\n"+name+"'s mother: \n"+output)
    
    # Adam's parents
    name = 'Adam'
    output = run(0, x, parent(x,name))
    print("\nName of "+name+ "'s parents:'")
    for item in output:
        print(item)
        
    # Wayne's grandparents
    name = 'Wayne'
    output = run(0, x, grandparent(x, name))
    print("\nName of "+name+ "'s grandparents:'")
    for item in output:
        print(item)
        
    # Megan's grandchildren
    name = 'Megan'
    output = run(0,x, grandparent(name, x))
    print("\nList of "+name+ "'s grandchildren:'")
    for item in output:
        print(item)
        
    
    # David's siblings
    name = 'David'
    output = run(0,x, sibling(x, name))
    siblings = [x for x in output if x != name]
    print("\nList of "+name+ "'s siblings:'")
    for item in output:
        print(item)
        
    # Tiffany's uncle
    name = 'Tiffany'
    name_father = run(0,x,father(x, name))[0]
    output = run(0,x,uncle(x,name))
    output = [x for x in output if x != name_father]
    print("\nList of "+name+ "'s uncles:'")
    for item in output:
        print(item)
        
    # All spouses
    a,b,c = var(), var(), var()
    output = run(0, (a,b), (father,a,c), (mother,b,c))
    print("\nList of all spouses: ")
    for item in output:
        print('Husband:', item[0], '<=> Wife:', item[1])
    
    
    


 List of John's children:'
William
Adam
David

William's mother: 
Megan

Name of Adam's parents:'
John
Megan

Name of Wayne's grandparents:'
John
Megan

List of Megan's grandchildren:'
Sophia
Julie
Chris
Wayne
Stephanie
Neil
Peter
Tiffany

List of David's siblings:'
William
Adam
David

List of Tiffany's uncles:'
William
Adam

List of all spouses: 
Husband: John <=> Wife: Megan
Husband: David <=> Wife: Olivia
Husband: William <=> Wife: Emma
Husband: Adam <=> Wife: Lily


In [17]:
# Analyzing Geography

from kanren import run, var, Relation, eq, fact 

# Initialize the relation
adjacent = Relation()
coastal = Relation()

# Open input files
file_coastal = 'C:/Users/Peyman/Desktop/Python - Practices/coastal_states.txt'
file_adjacent = 'C:/Users/Peyman/Desktop/Python - Practices/adjacent_states.txt'

# Read the file containg the coastal states
with open(file_coastal, 'r') as f:
    line = f.read()
    coastal_states = line.split(',')
    
# Add info to the facts
for state in coastal_states:
    fact(coastal, state)
    # fact is like an object that has 'predicate' that is the property of the object 

# Read the file containg the adjacent states
with open(file_adjacent, 'r') as f:
    adjlist = [line.strip().split(',') for line in f if line and line[0].isalpha()]
    
# Add info to the fact
for L in adjlist:
    head, tail = L[0], L[1:]
    for state in tail:
        fact(adjacent, head, state)
        
# Initialize the variables
x = var()
y = var()

# Testing the code
# Is Nevada adjacent to Lousisiana?
output = run(0, x, adjacent('Nevada', 'Louisiana'))
print('\nIs Nevada adjacent to Louisiana')
print ('Yes' if len(output) else 'No')

# States adjacent to Oregon
output = run(0,x, adjacent('Oregon', x))
print('\nList of states adjacent to Oregon:')
for item in output:
    print(item)
    
#States adjaent to Mississippi that are coastal
output = run(0, x, adjacent('Mississippi', x), coastal(x))
print('\nList of coastal states adjacent to Mississippi:')
for item in output:
    print(item)
    
# List of 'n' states that border a coastal state
n = 7
output = run(n, x, coastal(y), adjacent(x,y))
print('\nList of '+str(n) + ' states that boarder a coastal state:')
for item in output:
    print(item)
    
# List of states that are adjacent to two states
output = run(0, x, adjacent('Arkansas', x), adjacent('Kentucky', x))
print('\nList of states that are adjacent to Arkansas and Kentucky:')
for item in output:
    print(item)
    
    



Is Nevada adjacent to Louisiana
No

List of states adjacent to Oregon:
Idaho
Nevada
Washington
California

List of coastal states adjacent to Mississippi:
Alabama
Louisiana

List of 7 states that boarder a coastal state:
Tennessee
Massachusetts
Oregon
Pennsylvania
New York
Idaho
Maine

List of states that are adjacent to Arkansas and Kentucky:
Tennessee
Missouri


In [92]:
# Building a Puzzle
from kanren import *
from kanren import membero
from kanren.core import lall

from kanren import isvar,run,membero
from kanren.core import success,fail,goaleval,condeseq,eq,var

# Declare the variable 'people'
people = var()

# Define the ruels
rules = lall(
    # Steve's car is blue
    (membero, people, ('Steve', var(),'blue', var())),
    
    # Person who has a cat lives in Canada
    (membero, people, (var(), 'cat', var(), 'Canada')),
    #(membero(x, (var(), 'cat', var(), 'Canada')), people),

    # Mathew lives in US
    (membero, people, ('Mathew', var(), var(), 'USA')),

    # The person who has a black car lives in Australia
    (membero, people, (var(), var(), 'black', 'Australia')),

    #Jack has a cat
    (membero, people, ('Jack', 'cat', var(), var())), 

    # Alfered lives in Australia
    (membero, people, ('Alfered', var(), var(), 'Australia')),

    # Person who owns a dog lives in France
    (membero, people, (var(), 'dog', var(), 'France')),

    # Who has a rabbit?
    (membero, people, (var(), 'rabbit', var(), var()))
    )

# Run the solver
solutions = run(0, people, rules)

# Extract the output

output = [h for h in solutions[0] if 'rabbit' in h][0][0]

# Print the output
print('\n' + output + 'is the owner of the rabbit')
print('\nHere are all the details:')
attribs = ['Name', 'Pet', 'Color','Country']
print('\n'+ '\t\t'.join(attribs))
print('=' * 57)
for item in solutions[0]:
    print('')
    print('\t\t'.join([str(x) for x in item]))

IndexError: list index out of range